# 🔬 CLARISSA · SPE Benchmark Cross-Validation

**Real OPM Flow 2025.10 results** from Nordic VM simulation runs.

| Benchmark | Grid | Cells | Wells | Simulation Time |
|-----------|------|-------|-------|----------------|
| SPE1 | 10×10×3 | 300 | 2 | 10 years |
| SPE5 | 7×7×3 | 147 | 3 | 22 years |
| SPE9 | 24×25×15 | 9,000 | 26 | 900 days |
| SPE10 M1 | 100×1×20 | 2,000 | 2 | 8,000 days |

> PAL Architecture · ICE v2.1 · ADR-040

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wolfram-laube/clarissa/blob/main/notebooks/CLARISSA_SPE_Cross_Validation.ipynb)


In [ ]:
# Setup — works in Colab, Nordic VM Jupyter, and local
import json, os, sys, urllib.request
import numpy as np

DATA_URL = "https://raw.githubusercontent.com/wolfram-laube/clarissa/main/results/spe_benchmarks_real.json"

# Try local paths first
DATA_FILE = None
for path in [
    os.path.expanduser("~/projects/results/spe_benchmarks_real.json"),  # Nordic VM
    "spe_benchmarks_real.json",
    "../results/spe_benchmarks_real.json",
]:
    if os.path.exists(path):
        DATA_FILE = path
        break

if DATA_FILE is None:
    # Fetch from GitHub (public raw or with token for private)
    DATA_FILE = "/tmp/spe_benchmarks_real.json"
    try:
        # Try Colab secrets first
        from google.colab import userdata
        token = userdata.get('GITHUB_PAT')
        req = urllib.request.Request(DATA_URL, headers={"Authorization": f"token {token}"})
        urllib.request.urlretrieve(req.full_url, DATA_FILE)
    except:
        try:
            urllib.request.urlretrieve(DATA_URL, DATA_FILE)
        except:
            print("⚠ Upload spe_benchmarks_real.json manually or set GITHUB_PAT in Colab secrets")
            DATA_FILE = None

if DATA_FILE and os.path.exists(DATA_FILE):
    with open(DATA_FILE) as f:
        data = json.load(f)
    print(f"✅ Loaded {len(data['benchmarks'])} benchmarks:")
    for idx in data['index']:
        bm = data['benchmarks'][idx['id']]
        ts = bm['opm']['timesteps']
        print(f"   {idx['id']:8s} {idx['grid']:12s} {len(ts):3d} steps  {idx['description'][:55]}")
else:
    raise FileNotFoundError("No benchmark data — see instructions above")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# CLARISSA dark theme
C = {
    'bg': '#0a0e1a', 'surface': '#111827', 'text': '#e2e8f0',
    'cyan': '#22d3ee', 'purple': '#a78bfa', 'green': '#34d399',
    'amber': '#fbbf24', 'red': '#f87171', 'blue': '#60a5fa'
}

def cstyle(fig, axes):
    """Apply CLARISSA dark theme."""
    fig.patch.set_facecolor(C['bg'])
    for ax in (axes.flat if hasattr(axes, 'flat') else [axes]):
        ax.set_facecolor(C['surface'])
        ax.tick_params(colors=C['text'], labelsize=9)
        for attr in ('xaxis', 'yaxis'):
            getattr(ax, attr).label.set_color(C['text'])
        ax.title.set_color(C['text'])
        for s in ax.spines.values(): s.set_color('#2a3654')
        ax.grid(True, alpha=0.15, color='#4a5568')

BM_IDS = ['SPE1', 'SPE5', 'SPE9', 'SPE10']
print("🎨 CLARISSA theme ready")


## 1. Production Profiles
Field oil, water and gas rates — the primary cross-validation metric.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
cstyle(fig, axes)

for ax, bm_id in zip(axes.flat, BM_IDS):
    ts = data['benchmarks'][bm_id]['opm']['timesteps']
    g = data['benchmarks'][bm_id]['grid']
    t = [s['time_days']/365.25 for s in ts]
    
    oil  = [sum(w.get('oil_rate_m3_day',0) for w in s['wells']) for s in ts]
    water = [sum(w.get('water_rate_m3_day',0) for w in s['wells']) for s in ts]
    gas  = [sum(w.get('gas_rate_m3_day',0) for w in s['wells']) for s in ts]
    
    ax.plot(t, oil, color=C['green'], lw=2, label='Oil')
    if max(water) > 0.1: ax.plot(t, water, color=C['blue'], lw=2, label='Water')
    if max(gas) > 0.1:   ax.plot(t, gas, color=C['red'], lw=1.5, alpha=.7, label='Gas')
    
    ax.set_title(f"{bm_id} — {g['nx']}×{g['ny']}×{g['nz']}", fontsize=12, fontweight='bold')
    ax.set_xlabel('Time (years)'); ax.set_ylabel('Rate (m³/day)')
    ax.legend(fontsize=8, facecolor=C['surface'], edgecolor='#2a3654', labelcolor=C['text'])

fig.suptitle('CLARISSA · Field Production Rates (OPM Flow 2025.10)', color=C['cyan'], fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()


## 2. Pressure Evolution
Average, min and max reservoir pressure — injection support vs. depletion.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
cstyle(fig, axes)

for ax, bm_id in zip(axes.flat, BM_IDS):
    ts = data['benchmarks'][bm_id]['opm']['timesteps']
    t = [s['time_days']/365.25 for s in ts]
    pavg = [np.mean(s['cells']['pressure']) for s in ts]
    pmin = [np.min(s['cells']['pressure']) for s in ts]
    pmax = [np.max(s['cells']['pressure']) for s in ts]
    
    ax.fill_between(t, pmin, pmax, alpha=0.2, color=C['cyan'])
    ax.plot(t, pavg, color=C['cyan'], lw=2, label='Mean')
    ax.plot(t, pmin, color=C['cyan'], lw=.8, alpha=.4, ls='--')
    ax.plot(t, pmax, color=C['cyan'], lw=.8, alpha=.4, ls='--')
    ax.set_title(f"{bm_id} — Reservoir Pressure", fontsize=12, fontweight='bold')
    ax.set_xlabel('Time (years)'); ax.set_ylabel('Pressure (bar)')

fig.suptitle('CLARISSA · Pressure Evolution', color=C['cyan'], fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()


## 3. Oil Saturation Maps
2D slices at initial, mid and final timestep — gas front propagation and waterflood sweep.

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(16, 18))
cstyle(fig, axes)

for row, bm_id in enumerate(BM_IDS):
    bm = data['benchmarks'][bm_id]
    ts = bm['opm']['timesteps']
    nx, ny, nz = bm['grid']['nx'], bm['grid']['ny'], bm['grid']['nz']
    
    for col, ti in enumerate([0, len(ts)//2, -1]):
        ax = axes[row][col]
        so = np.array(ts[ti]['cells']['saturation_oil'])
        t_days = ts[ti]['time_days']
        
        if nz > 1 and ny > 1:
            so_3d = so.reshape(nz, ny, nx)
            k = nz // 2
            s2d = so_3d[k]; lbl = f"K={k+1}"
        elif ny == 1:
            s2d = so.reshape(nz, nx); lbl = "XZ"
        else:
            s2d = so.reshape(ny, nx); lbl = "XY"
        
        im = ax.imshow(s2d, cmap='RdYlGn', vmin=0, vmax=1, aspect='auto', origin='lower')
        ax.set_title(f"{bm_id} t={t_days:.0f}d ({lbl})", fontsize=10, fontweight='bold')
        plt.colorbar(im, ax=ax, shrink=0.8, label='So')

fig.suptitle('CLARISSA · Oil Saturation Maps', color=C['cyan'], fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout(); plt.show()


## 4. Permeability Distribution
Static rock property heterogeneity — from uniform layers (SPE1) to complex geology (SPE9/10).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
cstyle(fig, axes)

for ax, bm_id in zip(axes.flat, BM_IDS):
    bm = data['benchmarks'][bm_id]
    g = bm['grid']
    px = np.array(g['permx'])
    px_pos = px[px > 0]
    
    if len(np.unique(np.round(px, 1))) > 5:
        ax.hist(np.log10(px_pos), bins=50, color=C['purple'], alpha=0.7, edgecolor='none')
        ax.set_xlabel('log₁₀(Kx) [mD]')
        cv = np.std(px_pos)/np.mean(px_pos) if np.mean(px_pos)>0 else 0
        ax.text(0.95, 0.95, f'Cv={cv:.2f}\nn={len(px_pos)}\n{px_pos.min():.1f}–{px_pos.max():.0f} mD',
                transform=ax.transAxes, va='top', ha='right', fontsize=9,
                color=C['text'], bbox=dict(boxstyle='round', fc=C['surface'], ec='#2a3654'))
    else:
        u, c = np.unique(np.round(px, 1), return_counts=True)
        ax.bar([str(v) for v in u], c, color=C['purple'], alpha=0.7)
        ax.set_xlabel('Kx (mD)')
    
    ax.set_ylabel('Count')
    ax.set_title(f"{bm_id} — {g['nx']}×{g['ny']}×{g['nz']}", fontsize=12, fontweight='bold')

fig.suptitle('CLARISSA · Permeability Distribution', color=C['purple'], fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()


## 5. Recovery Factor
Oil recovery proxy from average saturation depletion.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
cstyle(fig, axes)

for ax, bm_id in zip(axes.flat, BM_IDS):
    ts = data['benchmarks'][bm_id]['opm']['timesteps']
    t = [s['time_days']/365.25 for s in ts]
    so = [np.mean(s['cells']['saturation_oil']) for s in ts]
    so0 = so[0] if so[0] > 0 else 1
    rf = [(1 - s/so0)*100 for s in so]
    
    ax.plot(t, rf, color=C['amber'], lw=2.5)
    ax.fill_between(t, 0, rf, alpha=0.15, color=C['amber'])
    ax.annotate(f'{rf[-1]:.1f}%', xy=(t[-1], rf[-1]), fontsize=11, fontweight='bold', color=C['amber'], ha='right')
    ax.set_title(f"{bm_id}", fontsize=12, fontweight='bold')
    ax.set_xlabel('Time (years)'); ax.set_ylabel('Recovery (%)')
    ax.set_ylim(0, max(rf)*1.3 if max(rf)>0 else 10)

fig.suptitle('CLARISSA · Recovery Factor', color=C['amber'], fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()


## 6. Summary

In [ ]:
print(f"{'='*78}")
print(f"{'Benchmark':8s} {'Grid':12s} {'Cells':>6s} {'Wells':>5s} {'Steps':>5s} {'P₀(bar)':>9s} {'Pf(bar)':>9s} {'ΔSo':>7s} {'RF%':>6s}")
print(f"{'='*78}")

for idx in data['index']:
    bm = data['benchmarks'][idx['id']]
    ts = bm['opm']['timesteps']
    g = bm['grid']
    nc = g['nx']*g['ny']*g['nz']
    p0 = np.mean(ts[0]['cells']['pressure'])
    pf = np.mean(ts[-1]['cells']['pressure'])
    so0 = np.mean(ts[0]['cells']['saturation_oil'])
    sof = np.mean(ts[-1]['cells']['saturation_oil'])
    rf = (1-sof/so0)*100 if so0>0 else 0
    
    grid_str = f"{g['nx']}×{g['ny']}×{g['nz']}"
    print(f"{idx['id']:8s} {grid_str:12s} {nc:6d} {len(ts[0]['wells']):5d} {len(ts):5d} {p0:9.1f} {pf:9.1f} {so0-sof:7.3f} {rf:6.1f}")

print(f"{'='*78}")
print(f"Source: OPM Flow 2025.10 · Nordic VM · PAL Architecture ICE v2.1 · ADR-040")
